In [ ]:
#This Python 3 environment comes with many helpful analytics libraries installed
#It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
#For example, here's several helpful packages to load
import numpy as np # linear algebra 
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

#Input data files are available in the read-only "../input/" directory
#For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


#import os 
#for dirname, _, filenames in os.walk('/kaggle/input'): 
#    for filename in filenames: 
#        print(os.path.join(dirname, filename))

#You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
#You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('/kaggle/input/widsdatathon2024-challenge1/training.csv')
test = pd.read_csv('/kaggle/input/widsdatathon2024-challenge1/test.csv')

# Let's impute 29 missing data points for Ozone, PM25 and NO2.
## I'll use XGBRegressor to predict the missing values

In [ ]:
train.drop(columns = ['metastatic_first_novel_treatment', 'metastatic_first_novel_treatment_type', 'patient_race', 'payer_type' , 'bmi'], inplace= True)
test.drop(columns = ['metastatic_first_novel_treatment', 'metastatic_first_novel_treatment_type', 'patient_race', 'payer_type' , 'bmi'], inplace= True)

In [ ]:
train['DiagPeriodL90D']=train['DiagPeriodL90D'].astype('category')

In [ ]:
pd.set_option('display.max_rows', 83)

In [ ]:
all_data = pd.concat([ train.drop('DiagPeriodL90D', axis= 1), test], axis = 0)

In [ ]:
ad_types = all_data.dtypes.to_dict()
for key in ad_types:
    if ad_types[key]=='O':
        all_data[key] = all_data[key].astype('category')


In [ ]:
#changing patient_zip3 into categorical variable

all_data['patient_zip3']=all_data['patient_zip3'].astype('category')
all_data.dtypes

In [ ]:
from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV
from sklearn.metrics import confusion_matrix, roc_auc_score
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder

In [ ]:
miss_ozone = all_data['Ozone'].isna()
miss_PM25 = all_data['PM25'].isna()
miss_NO2 = all_data['N02'].isna()

In [ ]:
ozone_data = all_data[miss_ozone]
PM25_data = all_data[miss_PM25]
NO2_data = all_data[miss_NO2]

In [ ]:
ozone_data.head()

In [ ]:
air_polution_traindata =  all_data[~miss_ozone]

In [ ]:
from sklearn.model_selection import StratifiedKFold
import xgboost as xgb

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV

In [ ]:
X = air_polution_traindata.drop(columns=['Ozone',  'patient_id'])
y = air_polution_traindata['Ozone']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.4, shuffle=True, random_state=66)

In [ ]:
gbm = xgb.XGBRegressor(enable_categorical = True)
reg_cv = GridSearchCV(gbm, {"colsample_bytree":[1.0],"min_child_weight":[1.0,1.2]
                            ,'max_depth': [3,4,6], 'n_estimators': [500,1000]}, verbose=3)
reg_cv.fit(X_train,y_train)

In [ ]:
reg_cv.best_params_

In [ ]:
gbm_ozone_pred = xgb.XGBRegressor(**reg_cv.best_params_, enable_categorical =  True)
gbm_ozone_pred.fit(X_train,y_train)

In [ ]:
predictions = gbm_ozone_pred.predict(X_test)
gbm_ozone_pred.score(X_test,y_test)

In [ ]:
ozone_pred = gbm_ozone_pred.predict(ozone_data.drop(columns=['Ozone','patient_id']))

In [ ]:
print(ozone_pred)

In [ ]:
X = air_polution_traindata.drop(columns=['N02',  'patient_id'])
y = air_polution_traindata['N02']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.4, shuffle=True, random_state=66)

In [ ]:
gbm = xgb.XGBRegressor(enable_categorical = True)
reg_cv = GridSearchCV(gbm, {"colsample_bytree":[1.0],"min_child_weight":[1.0,1.2]
                            ,'max_depth': [3,4,6], 'n_estimators': [500,1000]}, verbose=0)
reg_cv.fit(X_train,y_train)

In [ ]:
reg_cv.best_params_

In [ ]:
gbm_no2_pred = xgb.XGBRegressor(**reg_cv.best_params_, enable_categorical =  True)
gbm_no2_pred.fit(X_train,y_train)

In [ ]:
predictions = gbm_no2_pred.predict(X_test)
gbm_no2_pred.score(X_test,y_test)

In [ ]:
NO2_pred = gbm_no2_pred.predict(ozone_data.drop(columns=['N02','patient_id']))
print(NO2_pred)

In [ ]:
X = air_polution_traindata.drop(columns=['PM25',  'patient_id'])
y = air_polution_traindata['PM25']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.4, shuffle=True, random_state=66)

In [ ]:
gbm = xgb.XGBRegressor(enable_categorical = True)
reg_cv = GridSearchCV(gbm, {"colsample_bytree":[1.0],"min_child_weight":[1.0,1.2]
                            ,'max_depth': [3,4,6], 'n_estimators': [500,1000]}, verbose=0)
reg_cv.fit(X_train,y_train)

In [ ]:
reg_cv.best_params_

In [ ]:
gbm_pm25_pred = xgb.XGBRegressor(**reg_cv.best_params_, enable_categorical =  True)
gbm_pm25_pred.fit(X_train,y_train)

In [ ]:
predictions = gbm_pm25_pred.predict(X_test)
gbm_pm25_pred.score(X_test,y_test)

In [ ]:
PM25_pred = gbm_pm25_pred.predict(ozone_data.drop(columns=['PM25','patient_id']))
print(PM25_pred)

In [ ]:
missing_air_index = all_data[all_data['Ozone'].isna()].index.tolist()
print(missing_air_index)

In [ ]:
for count, idx in enumerate(missing_air_index):
    all_data.loc[idx, 'Ozone'] = ozone_pred[count]
    all_data.loc[idx, 'N02'] = NO2_pred[count]
    all_data.loc[idx, 'PM25'] = PM25_pred[count]

In [ ]:
all_data.to_csv('all_data_airpol.csv', index=False)

#I'll impute missing geo data [patient_state , Region, Division] using patient_zip3 column

In [ ]:
all_data.reset_index(inplace= True)

In [ ]:
missing_state = all_data[all_data['patient_state'].isna()]['patient_zip3']
missing_region = all_data[all_data['Region'].isna()]['patient_zip3']
missing_division = all_data[all_data['Division'].isna()]['patient_zip3']

In [ ]:
missing_region==missing_division

In [ ]:
dict_zip = {}
for ind in all_data.index:
    if all_data['patient_zip3'][ind] not in dict_zip.keys():
        dict_zip[all_data['patient_zip3'][ind]] = [all_data['patient_state'][ind], all_data['Division'][ind], all_data['Region'][ind]]
    
print(dict_zip)

In [ ]:
dict_zip[645] = ['MO', 'West North Central', 'Midwest']
dict_zip[157] = ['PA', 'Middle Atlantic', 'Northeast']
dict_zip[136] = ['NY', 'Middle Atlantic', 'Northeast']
dict_zip[181] = ['PA', 'Middle Atlantic', 'Northeast']
dict_zip[330] = ['FL', 'South Atlantic', 'South']
dict_zip[973] = ['OR', 'Pacific', 'West']

In [ ]:
for ind in all_data.index:
    if all_data['patient_state'][ind] is np.nan:
        all_data['patient_state'][ind] = dict_zip[all_data['patient_zip3'][ind]][0]
        
    if all_data['Division'][ind] is np.nan:
        all_data['Division'][ind] = dict_zip[all_data['patient_zip3'][ind]][1]
    if all_data['Region'][ind] is np.nan:
        all_data['Region'][ind] = dict_zip[all_data['patient_zip3'][ind]][2]

In [ ]:
all_data.to_csv('all_data_airpol_geo.csv', index=False)

In [ ]:
train = pd.read_csv('/kaggle/input/widsdatathon2024-challenge1/training.csv')
test = pd.read_csv('/kaggle/input/widsdatathon2024-challenge1/test.csv')

In [ ]:
payer_type = pd.concat([ train.drop('DiagPeriodL90D', axis= 1), test], axis = 0)

In [ ]:
payer_type.reset_index(inplace=True)

In [ ]:
all_data['payer_type']= payer_type['payer_type']

#Let's impute payer_type and patient_race

In [ ]:
cat_cols = ['payer_type', 'patient_state', 'patient_zip3', 'breast_cancer_diagnosis_code', 'breast_cancer_diagnosis_desc', 'metastatic_cancer_diagnosis_code', 'Region', 'Division']

In [ ]:
for col in cat_cols:
    all_data[col] = all_data[col].astype('category')

In [ ]:
miss_payer = all_data['payer_type'].isna() #
train_payer_data = all_data[~miss_payer] #complete data with payer_type
miss_payer_data = all_data[miss_payer] #data where payer type is missing and we have to impute it

In [ ]:
 all_data['payer_type'] = all_data['payer_type'].astype('category')

In [ ]:
train_payer_data.dtypes

In [ ]:
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder

In [ ]:
from sklearn.metrics import log_loss

In [ ]:
#cat_cols = ['payer_type', 'patient_state', 'patient_zip3', 'breast_cancer_diagnosis_code', 'breast_cancer_diagnosis_desc', 'metastatic_cancer_diagnosis_code', 'Region', 'Division']

In [ ]:
le = LabelEncoder()
#for col in cat_cols:
    #train_payer_data[col] = le.fit_transform(train_payer_data[col])

In [ ]:
#cat_cols_pred = ['patient_state', 'patient_zip3', 'breast_cancer_diagnosis_code', 'breast_cancer_diagnosis_desc', 'metastatic_cancer_diagnosis_code', 'Region', 'Division']

In [ ]:
#for col in cat_cols:
#    miss_payer_data[col] = le.fit_transform(miss_payer_data[col])

In [ ]:
all_data.head()

In [ ]:
X = train_payer_data.drop(columns = ['patient_gender', 'index', 'patient_id', 'payer_type'])
y = le.fit_transform(train_payer_data['payer_type'])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.4, stratify=y, shuffle=True, random_state=66)

In [ ]:
params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0, 0.5, 1],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5],
        'learning_rate' : [0.001, 0.01],
        'n_estimators' : [ 500, 1000]
        }

In [ ]:
xgb_new = XGBClassifier(objective='multi:softmax', enable_categorical=True)
skf = StratifiedKFold(n_splits=5, shuffle = True, random_state = 66)
random_new = RandomizedSearchCV(xgb_new, param_distributions=params, n_iter=80, scoring='neg_log_loss', n_jobs=4, cv=skf.split(X_train,y_train), verbose=0, random_state=66 )
random_new.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
payer_clf = XGBClassifier(**random_new.best_params_, enable_categorical=True)
payer_clf.fit(X_train, y_train)
test_pred = payer_clf.predict(X_test)
print(confusion_matrix(y_test, test_pred))

In [ ]:
all_data['payer_type'].isna().sum()

In [ ]:
miss_payer_data['payer_type'].isna().sum()

In [ ]:
payer_pred = payer_clf.predict(miss_payer_data.drop(columns = ['patient_gender', 'index', 'patient_id', 'payer_type']))
print(payer_pred)

In [ ]:
missing_payer_type_index = payer_type[payer_type['payer_type'].isna()].index.tolist()


In [ ]:
print(len(missing_payer_type_index), len(payer_pred))

In [ ]:
missing_payer_type_index[37]

In [ ]:
payer_type.loc[296, 'payer_type']

In [ ]:
all_data.loc[296, 'payer_type']

In [ ]:
all_data['payer_type'].dtypes

In [ ]:
payer_cat = ['COMMERCIAL', 'MEDICAID', 'MEDICARE ADVANTAGE']
payer_label = []
for label in payer_pred:
    payer_label.append(payer_cat[label])
    
    

In [ ]:
for count, idx in enumerate(missing_payer_type_index):
    all_data.loc[idx, 'payer_type'] = payer_label[count]
    



In [ ]:
all_data.isna().sum()

In [ ]:
all_data.to_csv('all_data_airpol_geo_payer.csv')

In [ ]:
all_data['patient_race'] = payer_type['patient_race']

In [ ]:
all_data.head(10)

In [ ]:
miss_race = all_data['patient_race'].isna()
train_miss_race = all_data[~miss_race]
pred_miss_race = all_data[miss_race]

In [ ]:
train_miss_race.columns

In [ ]:
X = train_miss_race.drop(columns=['index','patient_gender', 'patient_id', 'patient_race'])
y = le.fit_transform(train_miss_race['patient_race'])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.4, stratify=y, shuffle=True, random_state=66)

In [ ]:
xgb_new = XGBClassifier(objective='multi:softmax', enable_categorical=True)
skf = StratifiedKFold(n_splits=5, shuffle = True, random_state = 66)
random_new = RandomizedSearchCV(xgb_new, param_distributions=params, n_iter=80, scoring='neg_log_loss', n_jobs=4, cv=skf.split(X_train,y_train), verbose=0, random_state=66 )
random_new.fit(X_train, y_train)

In [ ]:
race_clf = XGBClassifier(**random_new.best_params_, enable_categorical=True)
race_clf.fit(X_train, y_train)
test_pred = race_clf.predict(X_test)
print(accuracy_score(y_test, test_pred))

print(confusion_matrix(y_test, test_pred))

In [ ]:
race_pred = race_clf.predict(pred_miss_race.drop(columns = ['patient_gender', 'index', 'patient_id', 'patient_race']))
print(race_pred)

In [ ]:
missing_race_index = payer_type[payer_type['patient_race'].isna()].index.tolist()

In [ ]:
print(len(missing_race_index), len(race_pred))

In [ ]:
payer_type['patient_race'] = payer_type['patient_race'].astype('category')

In [ ]:
payer_type['patient_race'].dtypes

In [ ]:
race_cat = ['Asian', 'Black', 'Hispanic', 'Other', 'White']
race_label = []
for label in race_pred:
    race_label.append(race_cat[label])

In [ ]:
for count, idx in enumerate(missing_race_index):
    all_data.loc[idx, 'patient_race'] = race_label[count]
    


In [ ]:
all_data.tail()

In [ ]:
all_data.to_csv('all_data_airpol_geo_payer_race.csv')

In [ ]:
sklearn.metrics.get_scorer_names()